# Exercício de limpeza de dados em Python

## Regras:

- Aparência: sol, nublado, chuva
- Temperatura: -130 ~ 130 F
- Umidade: 0 ~ 100
- Jogar: sim/não 
- Tratar valores NaN



In [2]:
### Importar bibliotecas
import pandas as pd
import seaborn as srn
import statistics as sts

## Importar dataframe

In [10]:
dataset = pd.read_csv('/home/diego/Desenvolvimento/Udemy_Formacao_Cientista_de_Dados/10.Prática em Python/dados/tempo.csv', sep=';')
dataset.shape

(14, 5)

In [8]:
dataset.head()

,Aparencia,Temperatura,Umidade,Vento,Jogar
0,sol,85,85.0,FALSO,nao
1,sol,80,90.0,VERDADEIRO,nao
2,nublado,83,86.0,FALSO,sim
3,chuva,70,NaN,FALSO,sim
4,chuva,68,80.0,FALSO,sim


In [9]:
dataset.tail()

,Aparencia,Temperatura,Umidade,Vento,Jogar
9,chuva,75,200.0,NaN,sim
10,sol,75,70.0,VERDADEIRO,sim
11,nublado,72,90.0,VERDADEIRO,sim
12,nublado,81,75.0,FALSO,sim
13,chuva,71,91.0,VERDADEIRO,nao
